# How to have chains

# 1. Create a simple chain

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import warnings
from langchain_core.output_parsers import StrOutputParser

warnings.filterwarnings("ignore")
from dotenv import load_dotenv, find_dotenv
import os
from langchain.chains import SimpleSequentialChain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

load_dotenv(find_dotenv(), override=True)
openai_api_key=os.getenv('OPENAI_API_KEY')


model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.5, model=model, max_tokens=4096)
tempate1 = """When did {person_name} born?
Date: this is the data of birth:"""
prompt_template1 = PromptTemplate(input_variables=['person_name'], template=tempate1)
chain1 = LLMChain(llm=llm, prompt=prompt_template1)



template2 = """You are a historian. Provide us five major events happened in a given date.
{date}
"""
prompt_template2 = PromptTemplate(input_variables=['date'], template=template2)

chain2 = LLMChain(llm=llm, prompt=prompt_template2)


simplechain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

output=simplechain.run('Barak Obama')

print(output)

# 2. Create a chain with an agent

In [13]:
from langchain.agents.load_tools import get_all_tool_names, load_tools
print(get_all_tool_names())

['requests', 'requests_get', 'requests_post', 'requests_patch', 'requests_put', 'requests_delete', 'terminal', 'sleep', 'wolfram-alpha', 'google-search', 'google-search-results-json', 'searx-search-results-json', 'bing-search', 'metaphor-search', 'ddg-search', 'google-lens', 'google-serper', 'google-scholar', 'google-finance', 'google-trends', 'google-jobs', 'google-serper-results-json', 'searchapi', 'searchapi-results-json', 'serpapi', 'dalle-image-generator', 'twilio', 'searx-search', 'merriam-webster', 'wikipedia', 'arxiv', 'golden-query', 'pubmed', 'human', 'awslambda', 'stackexchange', 'sceneXplain', 'graphql', 'openweathermap-api', 'dataforseo-api-search', 'dataforseo-api-search-json', 'eleven_labs_text2speech', 'google_cloud_texttospeech', 'reddit_search', 'news-api', 'tmdb-api', 'podcast-api', 'memorize', 'llm-math', 'open-meteo-api']


# Loading tools

In [ ]:
tools = load_tools(["ddg-search", "llm-math"], llm=llm)

# find the name and description of each tool
print(tools[0].name, tools[0].description)
print(tools[1].name, tools[1].description, "\n\n")

## tool name and description

#('duckduckgo_search',
# 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.')

from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", input_key="input", output_key="output",return_messages=True) 
agent = initialize_agent(
        agent="zero-shot-react-description",
        tools=tools,
        llm=llm,
        memory=memory,
        return_intermediate_steps= True, # Make sure you set it to True
        verbose=True
    )
print(agent.agent.llm_chain.prompt.template)

# Create a chain with an agent
This agent will be used to answer a list of embeddings questions and predefined QA, there will be a tool that will be watching the conversation and will trigger to create an order.
* create a prompt for the agent convesation
* create a retriever tool for the QA
* create a retriever tool for the inventory
* create a tool to identify the order